In [5]:
import pandas as pd
from collections import deque
import os

# File path for the state of consultant rotations
rotation_file = 'rotation_state.txt'

# Load the client data from the uploaded Excel file
file_path = 'Client_List.xlsx'  # Path to your file

# Read the data from column A to final population
client_data = pd.read_excel(file_path, usecols="A", skiprows=1, nrows=70, header=None)

# List of Consultants
consultant_names = ["AB", "KE", "CJ", "BL", "ZP",
                    "DS", "CT", "TZ"]


# Function to load the current rotation from the file or initialize it
def load_rotation_state():
    if os.path.exists(rotation_file):
        with open(rotation_file, 'r') as f:
            rotation_index = int(f.read().strip())
    else:
        rotation_index = 0
    return rotation_index

# Function to save the updated rotation index to the file
def save_rotation_state(rotation_index):
    with open(rotation_file, 'w') as f:
        f.write(str(rotation_index))

# Function to rotate consultants based on the saved state
def rotate_consultants(consultant_list, rotation_index):
    consultant_queue = deque(consultant_list)
    consultant_queue.rotate(-rotation_index)  # Rotate left by the saved index
    return list(consultant_queue)

# Load the current rotation state
rotation_index = load_rotation_state()

# Rotate consultant list so that each consultant gets a different set of clients based on the saved state
consultant_names = rotate_consultants(consultant_names, rotation_index)

# Evenly distribute the clients to each consultant
num_clients_per_consultant = len(client_data) // len(consultant_names)
remainder = len(client_data) % len(consultant_names)

# Create a dictionary to hold consultants and their assigned clients
consultant_assignment = {}

# Distribute the client list names accordingly
start_idx = 0
for i, consultant in enumerate(consultant_names):
    end_idx = start_idx + num_clients_per_consultant + (1 if i < remainder else 0)
    consultant_assignment[consultant] = client_data[start_idx:end_idx].values.flatten().tolist()
    start_idx = end_idx

# Display the assignments in the console
for consultant, clients in consultant_assignment.items():
    print(f"\n{consultant} is consulting:")
    for client in clients:
        print(f" - {client}")

# Write to a new Excel file with proper formatting and hiding gridlines
output_path = 'Weekly_Consultant_Client_Assignments_.xlsx'  # Output file name
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    # Create a workbook and worksheet
    workbook = writer.book
    worksheet = workbook.add_worksheet('Assignments')

    # Define formatting: bold title style for consultant names and autofit columns
    bold_title_format = workbook.add_format({
        'bold': True,
        'font_size': 14,  # Similar to title style
        'font_name': 'Calibri'
    })

    # Start adding data to the worksheet in columns
    col_widths = [0] * len(consultant_names)  # Track column widths for autofit

    col = 0  # Start from the first column
    row = 0  # Initialize row pointer
    
    # Set the number of columns for the distribution
    num_cols = len(consultant_names)
    
    for i, (consultant, clients) in enumerate(consultant_assignment.items()):
        # Write the consultant name in bold title style
        worksheet.write(row, col, consultant, bold_title_format)
        col_widths[col] = max(col_widths[col], len(consultant))  # Track column width

        # Write each client under the consultant's name
        for client in clients:
            row += 1
            worksheet.write(row, col, client)
            col_widths[col] = max(col_widths[col], len(client))  # Track column width

        # Move to the next column for the next consultant
        col += 1
        row = 0 if col < num_cols else row + len(clients) + 2  # Reset row for next consultant
        col = col % num_cols  # Wrap around to next column
    
    # Autofit the columns by setting column widths
    for i, width in enumerate(col_widths):
        worksheet.set_column(i, i, width + 2)  # Add some padding to the width

    # Turn off gridlines
    worksheet.hide_gridlines(2)

# Save the Excel file
print(f'\nConsultant assignments have been saved to {output_path}')

# Increment the rotation index and save it
rotation_index = (rotation_index + 1) % len(consultant_names)  # Ensure it wraps around
save_rotation_state(rotation_index)



ZP is consulting:
 - Client #001
 - Client #002
 - Client #003
 - Client #004
 - Client #005
 - Client #006
 - Client #007

DS is consulting:
 - Client #008
 - Client #009
 - Client #010
 - Client #011
 - Client #012
 - Client #013
 - Client #014

CT is consulting:
 - Client #015
 - Client #016
 - Client #017
 - Client #018
 - Client #019
 - Client #020
 - Client #021

TZ is consulting:
 - Client #022
 - Client #023
 - Client #024
 - Client #025
 - Client #026
 - Client #027
 - Client #028

AB is consulting:
 - Client #029
 - Client #030
 - Client #031
 - Client #032
 - Client #033
 - Client #034

KE is consulting:
 - Client #035
 - Client #036
 - Client #037
 - Client #038
 - Client #039
 - Client #040

CJ is consulting:
 - Client #041
 - Client #042
 - Client #043
 - Client #044
 - Client #045
 - Client #046

BL is consulting:
 - Client #047
 - Client #048
 - Client #049
 - Client #050
 - Client #051
 - Client #052

Consultant assignments have been saved to Weekly_Consultant_Client_